# Salarios - leng
[statter](https://plotly.com/python/line-and-scatter/)

In [1]:
import pandas as pd
import plotly.express as px  
from dash import Dash, dcc, html, Input, Output
import numpy as np
import PyCurrency_Converter

#crear un dataframe con toda la informacion de la encuesta
df21 = pd.read_csv ('../data/survey_results_public2021.csv', index_col = [0]) 
df20 = pd.read_csv ('../data/survey_results_public2020.csv', index_col = [0]) 


# El indice sera la columna con el ID de la respuesta


## 1. Preprocesar datos.
Se van a juntar los dos graficos de salarios en la misma funcion que preprocesa datos. Se reescribira la funcion del notebook salarios. 

In [2]:
def salario (df,anyo, opt):
    
    col = ""
    
    if opt == 0: col = "DevType"
    elif (opt == 1 and anyo==20): col = "LanguageWorkedWith"
    elif (opt == 1 and anyo==21): col = "LanguageHaveWorkedWith"
    
    
    if (anyo == 20): 
        #copia de df y se eliminan todas las columnas que tengan nan: Solo se seleccionan filas completas y se descartan las demas
        df4=df[[col,'ConvertedComp', 'YearsCodePro','Age1stCode']].dropna().copy()
         #para ponder usar el mismo callback en ambas opciones, hay que renombrar las columnas
        df4.rename(columns = {'ConvertedComp':'ConvertedCompYearly'}, inplace = True)
        df4.rename(columns = {'LanguageWorkedWith':'LanguageHaveWorkedWith'}, inplace = True)
        if (opt == 1): col = "LanguageHaveWorkedWith" #renombrar la variable col tambien.
        
    
    elif (anyo == 21): #Age1stCode en el df del 2021 son string en lugar de enteros.
         #copia de df y se eliminan todas las columnas que tengan nan: Solo se seleccionan filas completas y se descartan las demas
        df4=df[[col,'ConvertedCompYearly', 'YearsCodePro','Age1stCode']].dropna().copy()
        
        #crear diccionario con los cambios de valores
        edadesTransformadas = {"Younger than 5 years" : 4, "5 - 10 years" : 7, "11 - 17 years": 18, "18 - 24 years" : 21,
        "25 - 34 years" : 30, "35 - 44 years" : 40, "45 - 54 years": 50, "55 - 64 years" : 60, 
        "Older than 64 years": 70}
        
        #cambiar los valores.
        df4=df4.replace({"Age1stCode": edadesTransformadas})
        
    
    #resto es igual para ambos df y opciones
    
    # eliminar filas en anyos que tengan texto. 
    df4.drop(df4[df4['YearsCodePro'] == "Less than 1 year"].index, inplace = True) 
    df4.drop(df4[df4['YearsCodePro'] == "More than 50 years"].index, inplace = True)
    
    df4.drop(df4[df4['Age1stCode'] == "Younger than 5 years"].index, inplace = True) 
    df4.drop(df4[df4['Age1stCode'] == "Older than 85"].index, inplace = True)
    
    #convertir columnas a entero
    df4["Age1stCode"] = pd.to_numeric(df4["Age1stCode"])
    df4["YearsCodePro"] = pd.to_numeric(df4["YearsCodePro"])
    
    #separar filas con mas de un devtype y despues agruparlas
    # df.assign(**kargs) -> https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.assign.html
    df4=df4.assign(**{col:df4[col].str.split(';')}).explode(col).groupby(col).agg(avg_pro=('YearsCodePro', 'mean'), #nueva columna que calculara la media de yearscodepro
             avg_age=('Age1stCode','mean'), 
             median_money=('ConvertedCompYearly','median'), #nueva columna con mediana del salario
             # contar los diferentes tipos de devtype                                                                                    
             respuestas=(col,'count')).reset_index() #reiniciar el indice
    
    return df4

In [3]:
# salario (df20,20, 0)

## 2. Layout. 


In [4]:
app = Dash(__name__)
server = app.server #heroku
app.layout = html.Div([

    html.H1("Tipo de desarrollador", style={'text-align': 'center'}), #cabecero h1. Header
    
    #primera mini prueba con un menu desplegable.
    dcc.Dropdown(id="opt4",  
                 options=[ #el usuario va a ver las label.
                     {"label": "2020", "value": 2020},
                     {"label": "2021", "value": 2021}],
                 multi=False,
                 value=2020,
                 style={'width': "40%"}
                 ),
    
    html.Button('Button 1', id='hide-show', n_clicks=0),

    dcc.Graph(id='my_survey', figure={}) # graph container

])

# 3. Output que genera el grafico. 

En este caso, un [scatter](https://plotly.com/python/line-and-scatter/).

Cambiar "LanguageHaveWorkedWith" por Devtype en otro callback para generar el otro grafico. 

In [5]:
@app.callback(
    [Output(component_id='my_survey', component_property='figure'),
    Output('hide-show', 'children')],
    [Input(component_id='opt4', component_property='value'),
    Input(component_id='hide-show', component_property='n_clicks')])
def update_graph(opt4, n_clicks): #se pasan los dos input.
    
    df = pd.DataFrame()

    if (opt4 == 2020): 
        df= df20 
        anyo = 20
    else: 
        df = df21 
        anyo = 21

    fig =  px.scatter(salario (df,anyo,1), x= "avg_pro", y= "avg_age", size="respuestas", 
                      color="LanguageHaveWorkedWith",  hover_name="LanguageHaveWorkedWith", 
                      text = "LanguageHaveWorkedWith", hover_data=["median_money"],
                      labels={'avg_pro':'Years Coding Profesionaly (average)',
                              'avg_age':'Years since start coding (average)',
                              'respuestas':'# responses','median_money':'Median salary ($)', 
                              'LanguageHaveWorkedWith':'Language'},
                     color_discrete_sequence=px.colors.qualitative.Pastel,)
    
    fig.update_traces(textposition='top center') # texto que va encima de los puntitos

    #quitar color y grid del grafico
    fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)','paper_bgcolor': 'rgba(0, 0, 0, 0)',}) #fondo transparente
    
    
    bool_disabled = n_clicks % 2
    if bool_disabled:
        return fig, "Hide Leyend" # se devuelven los dos output
    else:
        fig.update_layout(showlegend=False)
        return fig, "Show Leyend"

## 4. run server

In [ ]:
app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
